# Collaborative Filtering with Neural Nets

In [1]:
# for data manipulation
import numpy as np
import pandas as pd
import os
import pickle

# use surprise for collaborative filtering
import torch
import torch.nn as nn
from torch.autograd import Variable

## Read in data

In [2]:
game_data_path = "data/neural_net_data/"
files = sorted(os.listdir(game_data_path))
with open(game_data_path + files[0], 'rb') as f:
    print(files[0])
    X, y = pickle.load(f, encoding='latin1')

2007-2008.pkl


In [16]:
X.shape

torch.Size([1236, 1512])

In [17]:
y.shape

torch.Size([1236, 1])

In [5]:
def trainValSplit(X, y):
    X = np.array(X)
    y = np.array(y)

    X = X[y > 0]
    y = y[y > 0]

    p = np.random.permutation(len(X))
    X = X[p]
    y = y[p]

    val = 0.2
    val = round(len(X) * val)
    X_val = X[:val]
    y_val = y[:val]
    X = X[val:]
    y = y[val:]
    
    return X, y, X_val, y_val

In [6]:
X, y, X_val, y_val = trainValSplit(X, y)

In [3]:
# BySeason trainValSplit
game_data_path = "data/neural_net_data/"
files = sorted(os.listdir(game_data_path))
with open(game_data_path + files[1], 'rb') as f:
    print(files[1])
    X_val, y_val = pickle.load(f, encoding='latin1')
    
X = X[y > 0]
y = y[y > 0]
X_val = X_val[y_val > 0]
y_val = y_val[y_val > 0]

2008-2009.pkl


In [4]:
y = torch.from_numpy(y[:,np.newaxis]).type(torch.FloatTensor)
X = torch.from_numpy(X.reshape((X.shape[0], -1))).type(torch.FloatTensor)
y_val = torch.from_numpy(y_val[:,np.newaxis]).type(torch.FloatTensor)
X_val = torch.from_numpy(X_val.reshape((X_val.shape[0], -1))).type(torch.FloatTensor)

In [5]:
# Split train/test:
print(y[1:10])

tensor([[212.],
        [222.],
        [202.],
        [201.],
        [193.],
        [157.],
        [183.],
        [166.],
        [201.]])


In [7]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_output):
        super(Net, self).__init__()
        
        # layer 1 fully connected 150 units
        self.lin1 = nn.Linear(n_feature, 1000)
        
        # layer 2 fully connected 50 units
        self.lin2 = nn.Linear(1000, 200)
        
        # layer 3 fully connected 1 unit (output)
        self.lin3 = nn.Linear(200, 50)
        
        self.lin4 = nn.Linear(50, n_output)
        
        # dropouts
        self.drop1 = nn.Dropout(0.5)
        self.drop2 = nn.Dropout(0.4)
        self.drop3 = nn.Dropout(0.25)

    def forward(self, x):
        # perform dropout on input vector embeddings
        x = self.drop1(x)
        x = F.relu(self.lin1(x))
        # x = self.drop2(F.relu(self.lin1(x)))
        x = F.relu(self.lin2(x))
        # x = self.drop3(F.relu(self.lin2(x)))
        x = F.relu(self.lin3(x))
        x = self.lin4(x)
        
        return x  

net = Net(n_feature=1512, n_output=1)     # define the network
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.000001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

# plt.ion()   # something about plotting

for t in range(5000):
    prediction = net(X)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    # Do validation loss
    with torch.no_grad():
        pred_val = net(X_val)
        loss_val = loss_func(pred_val, y_val)

    if t % 50 == 0:
        # plot and show learning process
        '''
        plt.cla()
        plt.scatter(x.data.numpy(), y.data.numpy())
        plt.plot(x.data.numpy(), prediction.data.numpy(), 'r-', lw=5)
        plt.text(0.5, 0, 'Loss=%.4f' % loss.data.numpy(), fontdict={'size': 20, 'color':  'red'})
        plt.pause(0.1)
        '''
        print("-----")
        print(t)
        print('Loss=%.4f' % loss.data.numpy())
        print('Val Loss=%.4f' % loss_val.data.numpy())

# plt.ioff()
# plt.show()

Net(
  (lin1): Linear(in_features=1512, out_features=1000, bias=True)
  (lin2): Linear(in_features=1000, out_features=200, bias=True)
  (lin3): Linear(in_features=200, out_features=50, bias=True)
  (lin4): Linear(in_features=50, out_features=1, bias=True)
  (drop1): Dropout(p=0.5)
  (drop2): Dropout(p=0.4)
  (drop3): Dropout(p=0.25)
)
-----
0
Loss=46056.3711
Val Loss=34173.3438
-----
50
Loss=1541.1873
Val Loss=1207.1206
-----
100
Loss=1093.1804
Val Loss=929.3924
-----
150
Loss=852.3702
Val Loss=823.7252
-----
200
Loss=725.7302
Val Loss=706.5421
-----
250
Loss=670.0817
Val Loss=628.3708
-----
300
Loss=930.2850
Val Loss=805.4080
-----
350
Loss=709.7932
Val Loss=663.7621
-----
400
Loss=729.2579
Val Loss=671.8440
-----
450
Loss=705.6008
Val Loss=655.4719
-----
500
Loss=675.0945
Val Loss=601.2825
-----
550
Loss=579.0130
Val Loss=560.0422
-----
600
Loss=662.9434
Val Loss=611.7088
-----
650
Loss=687.9087
Val Loss=616.5823
-----
700
Loss=658.9225
Val Loss=584.9807
-----
750
Loss=598.2576
Val L

KeyboardInterrupt: 